# Classifier les fleurs d'iris selon leur espèce grâce à un réseau de neurones

## Imports <a class="anchor" id="imports"></a>
Nous commençons par importer les paquets nécessaires pour ce notebook. 

In [1]:
# Imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

Nous chargeons également l'extension de TensorBoard pour les notebooks. 
Cette commande n'est pas nécessaire lorsque notre code Python n'est pas dans un notebook. 

In [2]:
%load_ext tensorboard

## Création du modèle

Nous utilisons ici un modèle séquentiel avec 3 couches complètement connectées. 
- Les deux premières couches contiennent chacune 8 neurones, et ont une fonction d'activation sigmoide ;
- La dernière couche contient 3 neurones, et a une fonction d'activation sigmoïde. Ainsi, la sortie de cette couche sera constituée de 3 nombres entre 0 et 1 et dont la somme vaut 1, qui pourront être interprétés comme des probabilités. 

Nous n'ajoutons pas d'extinction de neurones à ce réseau, car chaque couche ne contient que peu de neurones. Eventuellement, si nous voyons que le réseau est en situation de surapprentissage, nous pourrons ajouter une couche d'extinction de neurones, avec un taux d'extinction faible. 

In [3]:
model = models.Sequential()

model.add(layers.Dense(8, activation='relu', input_shape=(4,)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

## Chargement et séparation des données


In [4]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)

Vérifions la dimension de chacun de ces éléments. 

In [5]:
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(90, 4) (60, 4) (90,) (60,)


X_train a 90 lignes et 4 colonnes : il contient donc 90 exemples, chacun avec 4 caractéristiques. 

X_test a 60 lignes et 4 colonnes : cet ensemble contient 60 exemples, chacun ayant lui aussi 4 caractéristiques. 

Les vecteurs y_train et y_test ont pour taille respective 90 et 60, ce qui est cohérent avec la taille des matrices X_train et X_test. 

## Conversion des étiquettes en vecteurs au format un parmi n

In [6]:
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_test_cat = tf.keras.utils.to_categorical(y_test)

Vérifions une nouvelle fois la dimension de ces éléments. 

In [7]:
print (y_train_cat.shape, y_test_cat.shape)

(90, 3) (60, 3)


Comme attendu, les y_train_cat et y_test_cat sont maintenant des matrices avec 3 colonnes, une pour chaque classe considérée.

## Compilation du modèle
Nous indiquons à Keras que notre modèle devra être optimisé en utilisant les éléments suivants :
- la fonction d'optimisation d'Adam avec un taux d'aprentissage égal à 1e-3 ;
- une fonction de perte d'entropie croisée ;
- et en calculant la justesse à chaque itération. 

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

## Définition des fonctions de rappel

Nous utilisons la fonction de rappel de TensorBoard. 

In [9]:
log_dir = "log/fit/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

# Entraînement du modèle
Nous pouvons maintenant lancer l'entraînement du modèle, avec les paramètres suivants :
- les données d'entraînement sont X_train et y_train_cat ;
- les données de validation, utilisées pour l'affichage, sont X_test et y_test_cat, 
- nous demandons à la fonction d'optimisation de réaliser 100 époques ;
- nous utilisons une taille de lot très réduite (8 exemples par lot), car notre jeu de données ne contient que peu d'exemples ;
- et nous demandons à la fonction d'optimisation d'exécuter la fonction de rappel de TensorBoard à chaque itération, afin de sauvegarder les informations d'apprentissage. 

In [10]:
model.fit(X_train, y_train_cat, validation_data=(X_test, y_test_cat), epochs=50, batch_size=8, callbacks=[tensorboard_callback])

Epoch 1/50
12/12 [==============================] - 4s 56ms/step - loss: 2.4818 - accuracy: 0.3333 - val_loss: 1.9990 - val_accuracy: 0.3333
Epoch 2/50
12/12 [==============================] - 0s 7ms/step - loss: 1.7463 - accuracy: 0.3444 - val_loss: 1.4512 - val_accuracy: 0.5000
Epoch 3/50
12/12 [==============================] - 0s 7ms/step - loss: 1.2876 - accuracy: 0.6222 - val_loss: 1.0880 - val_accuracy: 0.6667
Epoch 4/50
12/12 [==============================] - 0s 10ms/step - loss: 0.9956 - accuracy: 0.6667 - val_loss: 0.8971 - val_accuracy: 0.6333
Epoch 5/50
12/12 [==============================] - 0s 15ms/step - loss: 0.8380 - accuracy: 0.7889 - val_loss: 0.8309 - val_accuracy: 0.6333
Epoch 6/50
12/12 [==============================] - 0s 9ms/step - loss: 0.8250 - accuracy: 0.6667 - val_loss: 0.8134 - val_accuracy: 0.6667
Epoch 7/50
12/12 [==============================] - 0s 6ms/step - loss: 0.7847 - accuracy: 0.6667 - val_loss: 0.7759 - val_accuracy: 0.6667
Epoch 8/50
12/12 